In [19]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import VOCSegmentation
from torchvision import models
import matplotlib.pyplot as plt
from PIL import Image

In [43]:
# Instantiate your weakly supervised dataset
# YourWeaklySupervisedDataset should provide images and their weak annotations
# You need to implement this dataset class

# Create a DataLoader for training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
train_dataset = VOCSegmentation(root='./data', year='2012', image_set='train', download=True, transform=transform)
pixel_data = train_dataset[0][1].load()
width, height = train_dataset[0][1].size
print("Pixel values:")
for y in range(height):
    for x in range(width):
        pixel_value = pixel_data[x, y]
        if pixel_value > 0:
            print(pixel_value)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Pixel values:
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
2

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
1
1
1
1
1
255
255
255
255
255
255
255
255
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
255
255
255
255
255
255
255
255
255
255
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
1
1
1
255
255
255
255
255
255
255
255
255


15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
15
15
15
15
15
15
15
15
15
15
255
255
255
255
255
255
255
255
25

In [ ]:
# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        # Define your encoder (pretrained on ImageNet)
        # For simplicity, let's use a ResNet18 as an example
        self.encoder = nn.Sequential(*list(models.resnet18(pretrained=True).children())[:-2])
        # Modify the decoder according to your needs
        # This is a simplified version
        self.decoder = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, num_classes, kernel_size=1)
        )

    def forward(self, x):
        # Forward pass through the encoder
        x = self.encoder(x)
        print(x.shape)
        # Forward pass through the decoder
        x = self.decoder(x)
        return x

In [34]:
# Set your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
model = UNet(num_classes=21).to(device)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [37]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, annotations in train_loader:
        images, annotations = images.to(device), annotations.to(device)

        # Forward pass
        print(images.shape)
        outputs = model(images)

        # Compute your loss based on the weak annotations
        print(outputs.shape)
        print(annotations)
        loss = criterion(outputs, annotations)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save or use the trained model for inference
torch.save(model.state_dict(), 'weakly_supervised_segmentation_model.pth')

torch.Size([16, 3, 224, 224])
torch.Size([16, 512, 7, 7])
torch.Size([16, 21, 7, 7])
tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0588, 0.0588, 0.0588,  ..., 0.0000, 0.0000, 0.0000],
          [0.0588, 0.0588, 0.0588,  ..., 0.0000, 0.0000, 0.0000],
          [0.0588, 0.0588, 0.0588,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0745, 

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4